# <center><u>RUN SWING SOLEIL octobre24</u></center>

Ce notebook extrait les informations des fichiers h5 contenus dans un dossier. 
- Extrait dans un pandas dataframe le nom du fichier, sa transmission, son type, le sample name est utilisé pour determiner le 
canal et la position dans le cas où il est au format S_C_X_SF_X_P_X (S=Sample, R= ref, C= n°channel du capillaire,P=n°position dans le capillaire ,
SF= Stop flow LF=LostFlow )
- Associe un echantillon à sa référence dans un pandas datadrame suivant si le sample name commence par S ou F (asso)
- Calcule les integrations corrigées à partir de l'association sample / ref
- crée une vidèo à partir de l'ensemble des images

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import glob
import os
import SwingTools as st
import VideoSwingTools as vst

- Dossier de travail

In [ ]:
folder = 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light'

# kin21, 37 et 23 et 24 si
#############  FAIT: 
#folder = 'D:/SWING/to_do/kin_21_au_500tes'
#folder = 'D:/SWING/to_do/kin_37_au_62tes_s'#
#folder = 'D:/SWING/to_do/kin_23_au_125tes_rt'
#folder = 'D:/SWING/to_do/kin_23_au_125tes_rt'
#############  A Faire: 
#folder = 'D:/SWING/to_do/kin_21_au_500tes'
############
#folder = 'D:/SWING/to_do/kin_37_au_62tes_s'
# folder = 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/dryed_cappilary/'
# folder = 'D:/swing/kin_11_ag_55deg'
# folder = 'D:/SWING/to_do/kin_23_au_125tes_rt'
#folder = 'D:/SWING/to_do/kin_37_au_62tes_s'
#folder = 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/dryed_cappilary/'
#folder = 'T:/LPCNO/NCO/Manips/DATA_SAXS/2024_10_11_SWING_Rohan/20240444/2024/Run4/rodriguez_manipOK/kin_21_au_500tes'
#folder = 'T:/LPCNO/NCO/Manips/DATA_SAXS/2024_10_11_SWING_Rohan/20240444/2024/Run4/rodriguez_manipOK/kin_24_au_250tes_rt'
#folder = 'D:/SWING/to_do/kin_44_ag_80_tes480_s'
#folder = 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light'
#folder = 'D:/SWING/to_do/kin_6'



- Extraction des dataframe et csv. 
    - df pour l'extraction des données de tous les fichiers 
    - asso pour les associations fichiers de mesure et de référence

In [3]:
df, asso = st.create_info_file(folder)
#si on modifie le dataframe manuellement:
#asso = pd.read_csv(folder+'_samples_ref_associated.csv')

2024-10-12 11:21:04


In [4]:
asso.head()

,File_sample,Sample name_sample,Mean Transmission_sample,Type_sample,Channel,Position,Time_sample,Flow_sample,File_ref,Sample name_ref,Mean Transmission_ref,Type_ref,Time_ref,Flow_ref,Time_ref_adjusted,File num.
0,rodriguez_04893_2024-10-12_11-48-45.h5,S_C_1_P_1,0.633914,sample,1,1,2024-10-12 11:48:45,unknown,rodriguez_04823_2024-10-12_11-21-04.h5,R_C_1_P_1,0.774261,ref,2024-10-12 11:21:04,unknown,00:00:00,04823
1,rodriguez_04896_2024-10-12_11-49-15.h5,S_C_1_P_4,0.629522,sample,1,4,2024-10-12 11:49:15,unknown,rodriguez_04826_2024-10-12_11-21-37.h5,R_C_1_P_4,0.772646,ref,2024-10-12 11:21:37,unknown,00:00:33,04826
2,rodriguez_04931_2024-10-12_11-55-10.h5,S_C_2_P_4,0.616391,sample,2,4,2024-10-12 11:55:10,unknown,rodriguez_04861_2024-10-12_11-27-30.h5,R_C_2_P_4,0.766370,ref,2024-10-12 11:27:30,unknown,00:06:26,04861
3,rodriguez_04962_2024-10-12_12-00-21.h5,S_C_2_P_35,0.611637,sample,2,35,2024-10-12 12:00:21,unknown,rodriguez_04892_2024-10-12_11-32-41.h5,R_C_2_P_35,0.765898,ref,2024-10-12 11:32:41,unknown,00:11:37,04892


In [5]:
df.head()

,File,Sample name,Mean Transmission,Type,Channel,Position,Time,Flow
0,rodriguez_04823_2024-10-12_11-21-04.h5,R_C_1_P_1,0.774261,ref,1,1,2024-10-12 11:21:04,unknown
1,rodriguez_04826_2024-10-12_11-21-37.h5,R_C_1_P_4,0.772646,ref,1,4,2024-10-12 11:21:37,unknown
2,rodriguez_04861_2024-10-12_11-27-30.h5,R_C_2_P_4,0.766370,ref,2,4,2024-10-12 11:27:30,unknown
3,rodriguez_04892_2024-10-12_11-32-41.h5,R_C_2_P_35,0.765898,ref,2,35,2024-10-12 11:32:41,unknown
4,rodriguez_04893_2024-10-12_11-48-45.h5,S_C_1_P_1,0.633914,sample,1,1,2024-10-12 11:48:45,unknown


In [6]:
print(len(df))
print(len(asso))

10
4


- Calcul des corrections d'intensité des integrations et sauvegarde des fichiers correspondant

In [7]:
#maskfile = 'mask.edf'
maskfile = 'macro_mask/mask_waxs_eiger_ruiz_oct24.edf'
# coef = 0.0181 
#coef =0.0308
coef = 1 
st.integration_correction(folder,asso,maskfile,coef)

C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light\S_C_2_P_35_04962\positions/@pos_@S_C_2_P_35@04962.png
File: 4 on 4 


- récupération des fichiers images dans les dossier de chaque échantillon

In [8]:
vst.copy_png_files(folder)

- Création de grilles de 4 images avec eiger, basler, integration et position

In [9]:
vst.create_grids_from_paths(folder)

{'S_C_1_P_1': {'basler': [{'file_path': 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light\\copied_png_files\\@basler@S_C_1_P_1@04893.png', 'file_num': '04893'}], 'eiger': [{'file_path': 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light\\copied_png_files\\@eiger@S_C_1_P_1@04893.png', 'file_num': '04893'}], 'corr_integration_': [{'file_path': 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light\\copied_png_files\\@corr_integration_@S_C_1_P_1@04893.png', 'file_num': '04893'}], 'pos_': [{'file_path': 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light\\copied_png_files\\@pos_@S_C_1_P_1@04893.png', 'file_num': '04893'}]}, 'S_C_1_P_4': {'basler': [{'file_path': 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_trai

- Conversion en video

In [10]:
vst.convert_video(folder)

C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light\grid_images
Moviepy - Building video C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light\video.mp4.
Moviepy - Writing video C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light\video.mp4



Moviepy - Done !
Moviepy - video ready C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/kin21_light\video.mp4
Finished.
